In [1]:
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
from bs4 import BeautifulSoup

### 1. Scraping Book Links

In [2]:
df = pd.DataFrame()

for i in tqdm('a b c d e f g h i j k l m n o p q r s t u v w x y z other'.split(' ')):
  link = 'https://www.gutenberg.org/browse/titles/' + i

  r = requests.get(link)
  soup = BeautifulSoup(r.text, 'html.parser')

  df_ = pd.DataFrame([[title.text, title.find('a').get('href')] for title in soup.find_all('h2')], columns = ['title','link'])
  df = pd.concat((df_,df))

100%|██████████| 27/27 [00:42<00:00,  1.59s/it]


### 2. Scraping Book Details

In [6]:
def download(st,nd):
  data = []
  for i in tqdm(df['link'].values[st:nd]):
    try:
      link = 'https://www.gutenberg.org/' + i

      r = requests.get(link)
      soup = BeautifulSoup(r.text, 'html.parser')
      soup = soup.find('table', class_ = 'bibrec')
      sp = soup.find_all('tr')[0]

      if sp.find('th').text == 'Author':
        author = sp.find('a').text
        author_id = sp.find('a').get('href')


      df_ = pd.read_html(link)[-1]
      df_.rename(columns = {'0':'author_name','1':'author_id'}, inplace = True)

      data.append([author,author_id])
      df_.to_csv('books/' + link.split('/')[-1] + '.csv')
    except:
      data.append([np.nan,np.nan])

In [7]:
p1 = multiprocessing.Process(target = download, args = (1 , 12000))
p2 = multiprocessing.Process(target = download, args = (12000 , 24000))
p3 = multiprocessing.Process(target = download, args = (24000 , 36000))
p4 = multiprocessing.Process(target = download, args = (36000 , 48000))
p5 = multiprocessing.Process(target = download, args = (48000 , 60000))
p6 = multiprocessing.Process(target = download, args = (60000 , 72000))
p7 = multiprocessing.Process(target = download, args = (72000 , 84000))
p8 = multiprocessing.Process(target = download, args = (84000 , len(df)))

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()
p6.start()
p7.start()
p8.start()

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()
p6.join()
p7.join()
p8.join()

100%|██████████| 14150/14150 [4:41:39<00:00,  1.19s/it]


In [8]:
!gzip books

gzip: books is a directory -- ignored


In [9]:
import shutil

shutil.make_archive('books', 'zip', 'books')

'/content/books.zip'

In [10]:
df

,title,link
0,"$1,000 a Plate (English)",/ebooks/50921
1,"$1,000 a Plate (English)",/ebooks/50921
2,The $100 Prize Essay on the Cultivation of the...,/ebooks/25905
3,The $100 Prize Essay on the Cultivation of the...,/ebooks/25905
4,"The $30,000 Bequest, and Other Stories (English)",/ebooks/9003
...,...,...
5663,"Aztec Ruins National Monument, New Mexico (Eng...",/ebooks/46946
5664,The Aztec Treasure-House (English),/ebooks/21618
5665,Azul...Obras Completas Vol. IV (Spanish),/ebooks/52894
5666,Azul...Obras Completas Vol. IV (Spanish),/ebooks/52894


In [11]:
df.to_csv('data.csv', index = False)